In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


#Reading Data from CSV Files
temperature_readings = pd.read_csv('data/data.csv') #original dataset for temp readings 
temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


weather_readings = pd.read_csv('data/weatherData.csv') #orignal dataset for weather readings

# Converting date and time in temperature_readings to UNIX timestamp for comparision 
temperature_readings['datetime_str'] = temperature_readings['date'] + ' ' + temperature_readings['time']
temperature_readings['datetime'] = temperature_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
temperature_readings['unix_timestamp'] = temperature_readings['datetime'].apply(lambda x: int(x.timestamp()))

# Converting date and time in weather_readings to UNIX timestamp for comparision 
weather_readings['datetime_str'] = weather_readings['date'] + ' ' + weather_readings['time']
weather_readings['datetime'] = weather_readings['datetime_str'].apply(lambda x: datetime.strptime(x, "%a %b %d %Y %I:%M:%S %p"))
weather_readings['unix_timestamp'] = weather_readings['datetime'].apply(lambda x: int(x.timestamp()))

weather_result_col = [col for col in weather_readings.columns if "result" in col.lower()]

# Merging both data for it to be on the same time
merged_data = pd.merge_asof(
    temperature_readings,  # Left DataFrame
    weather_readings[['unix_timestamp']+weather_result_col],      # Right DataFrame
    on='unix_timestamp',   # Key column
    direction='nearest'    # Match the nearest time
)

datetime_string = merged_data['date'] + " " + merged_data["time"]
merged_data["ISO_formatted_datetime"] = pd.to_datetime(
    datetime_string,
    format="%a %b %d %Y %I:%M:%S %p"
)


# #Columns for lorWan Sensors
temperature_col = [
    col for col in merged_data.columns 
    if "lorawan_readings" in col.lower() and "temperature" in col.lower()
]

humidity_col = [
    col for col in merged_data.columns 
    if "humidity" in col.lower() and "lorawan_readings" in col.lower()
]

co2_col = [
    col for col in merged_data.columns 
    if "co2" in col.lower() and "lorawan_readings" in col.lower()
]

sensors_to_keep = ["Sensor_1", "Sensor_3", "Sensor_6"]
sensors_col = [col for col in merged_data.columns if any(sensor in col for sensor in sensors_to_keep)]

weather_cols_to_keep = ["weather_status","weather_temp","weather_humidity"]
weather_col = [col for col in merged_data.columns if any(weathercol in col for weathercol in weather_cols_to_keep)]

#adding avg temp humid and co2
merged_data['avg_temperature'] = merged_data[temperature_col].mean(axis=1)
merged_data['avg_humidity'] = merged_data[humidity_col].mean(axis=1)
merged_data['avg_co2'] = merged_data[co2_col].mean(axis=1)

avg_col = [
    col for col in merged_data.columns
    if "avg" in col.lower()
]

#Energy(power,energy,current) Data
energy_data = merged_data[["ISO_formatted_datetime"]+ sensors_col]
energy_data.columns = energy_data.columns.str.replace(
    r"Energy_Readings.Sensor_1\.(Current|Energy|Power)", "compressor_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_3\.(Current|Energy|Power)", "fancoil_1_\\1", regex=True
).str.replace(
    r"Energy_Readings.Sensor_6\.(Current|Energy|Power)", "fancoil_2_\\1", regex=True
)

#indoor Data
indoor_data = merged_data[["ISO_formatted_datetime"] + temperature_col + humidity_col + co2_col + avg_col]


#Weather data
weather_data = merged_data[["ISO_formatted_datetime"]+ weather_col]
weather_data.columns = weather_data.columns.str.replace(
    r"result.weather_status", "weather_status", regex=True
).str.replace(
    r"result.weather_temp", "weather_temp", regex=True
).str.replace(
    r"result.weather_humidity", "weather_humidity", regex=True
)


#merging all the needed data
energy_indoor_merged = pd.merge(energy_data, indoor_data, on='ISO_formatted_datetime', how='inner')
final_merged_data = pd.merge(energy_indoor_merged, weather_data, on='ISO_formatted_datetime', how='inner')

final_merged_data['total_energy'] = (
    final_merged_data['compressor_Energy'] +
    final_merged_data['fancoil_1_Energy'] +
    final_merged_data['fancoil_2_Energy']
)

final_merged_data['total_power'] = (
    final_merged_data['compressor_Power'] +
    final_merged_data['fancoil_1_Power'] +
    final_merged_data['fancoil_2_Power']
)

final_merged_data['total_current'] = (
    final_merged_data['compressor_Current'] +
    final_merged_data['fancoil_1_Current'] +
    final_merged_data['fancoil_2_Current']
)
final_merged_data['hour'] = pd.to_datetime(final_merged_data['ISO_formatted_datetime']).dt.hour
final_merged_data['day_of_week'] = pd.to_datetime(final_merged_data['ISO_formatted_datetime']).dt.dayofweek

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

C:\Users\chenj\AppData\Local\Temp\ipykernel_3972\850930544.py:20: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  temperature_readings = temperature_readings.interpolate(method='linear')#fill in null values


In [3]:
def create_target_col(df):
    comfort_condition = (
        (df['avg_co2'] < 1000) &  # CO2 level
        (40 <= df['avg_humidity']) & (df['avg_humidity'] <= 60)
    )
    energy_eff_condtion =(
        (df['total_power'] < 10) & 
        (df['total_current'] < 13)
    )

    comfortable_temp = df.loc[comfort_condition & energy_eff_condtion, 'avg_temperature'].median()

    power_weight = 1 - (df['total_power'] / df['total_power'].max())
    current_weight = 1 - (df['total_current'] / df['total_current'].max())
        
    '''
    Target temperature calculation:
    1. If comfort conditions are met, use the actual temperature
    2. Otherwise, calculate a weighted temperature considering:
        70% of the comfortable mean temperature
        30% of the actual temperature
        Adjusted by power and current consumption weights
    '''
    target = np.where(
        comfort_condition, 
        df['avg_temperature'], 
        0.7 * comfortable_temp + 0.3 * df['avg_temperature'] * (power_weight + current_weight) / 2
    )
    return target
    
     
target = create_target_col(final_merged_data)
final_merged_data['optimal_temp'] = target


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


# Relevant columns for input
input_features = [
    'compressor_Current', 'compressor_Energy', 'compressor_Power',
    'fancoil_1_Current', 'fancoil_1_Energy', 'fancoil_1_Power',
    'fancoil_2_Current', 'fancoil_2_Energy', 'fancoil_2_Power',
    'avg_temperature', 'avg_humidity', 'avg_co2',
    'weather_temp', 'weather_humidity', 'total_energy', 
    'total_power', 'total_current', 'hour', 'day_of_week'
]

# Adding one-hot encoding for weather_status
categorical_features = ['weather_status']

# Preprocessing
X = final_merged_data[input_features + categorical_features]
y = final_merged_data['optimal_temp']  # Replace 'target_temp' with the actual column name for the target

# Transform categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), input_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

X = preprocessor.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Neural network
model = Sequential([
# removing dropout since our dataset very less
    Dense(32, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)),
    
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),

#     BatchNormalization(),

#     # Second layer
#     Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
#     BatchNormalization(),

#     # Third layer
#     Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
#     BatchNormalization(),

    # Output layer
    Dense(1, activation='linear')  # Predicting a continuous value
])


# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train, 
    epochs=500, 
    batch_size=8, 
    validation_split=0.3, 
    verbose=1,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Absolute Error: {mae}")#lower the better

# Make predictions
predictions = model.predict(X_test)

model.save("models/optimal_temp_NN_model.h5")


Epoch 1/500


C:\Users\chenj\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


127/127 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 629.5652 - mae: 25.0276 - val_loss: 421.4310 - val_mae: 20.3605
Epoch 2/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 282.4111 - mae: 15.8668 - val_loss: 19.0154 - val_mae: 3.3670
Epoch 3/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 16.5476 - mae: 3.1077 - val_loss: 7.4874 - val_mae: 1.9551
Epoch 4/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.1685 - mae: 1.8394 - val_loss: 5.4973 - val_mae: 1.5236
Epoch 5/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.6237 - mae: 1.6387 - val_loss: 4.8039 - val_mae: 1.3301
Epoch 6/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8023 - mae: 1.3312 - val_loss: 4.1393 - val_mae: 1.1765
Epoch 7/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3079 - mae: 1.3282 - val_loss: 3.7442 - val_mae: 1.0982
Epoch 8/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.1846 - mae: 1.1630 - val_loss: 3.4248 - val_mae: 1.0426
Epoch 9/500
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/s

In [ ]:
# Extract training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_mae = history.history['mae']
val_mae = history.history['val_mae']

# Plot the metrics
epochs = range(1, len(train_loss) + 1)

# Loss plot
plt.figure(figsize=(24, 12))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()

# MAE plot
plt.subplot(1, 2, 2)
plt.plot(epochs, train_mae, label='Training MAE')
plt.plot(epochs, val_mae, label='Validation MAE')
plt.title('Mean Absolute Error (MAE) Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()